In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import json
import requests

from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    MimeType,
    MosaickingOrder,
    SentinelHubRequest,
    SentinelHubStatistical,
    SHConfig,
)

In [2]:
config = SHConfig("cdse")

In [3]:
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"

In [4]:
for collection in DataCollection:
    print(collection)

DataCollection.SENTINEL2_L1C
DataCollection.SENTINEL2_L2A
DataCollection.SENTINEL1
DataCollection.SENTINEL1_IW
DataCollection.SENTINEL1_IW_ASC
DataCollection.SENTINEL1_IW_DES
DataCollection.SENTINEL1_EW
DataCollection.SENTINEL1_EW_ASC
DataCollection.SENTINEL1_EW_DES
DataCollection.SENTINEL1_EW_SH
DataCollection.SENTINEL1_EW_SH_ASC
DataCollection.SENTINEL1_EW_SH_DES
DataCollection.DEM
DataCollection.DEM_MAPZEN
DataCollection.DEM_COPERNICUS_30
DataCollection.DEM_COPERNICUS_90
DataCollection.MODIS
DataCollection.LANDSAT_MSS_L1
DataCollection.LANDSAT_TM_L1
DataCollection.LANDSAT_TM_L2
DataCollection.LANDSAT_ETM_L1
DataCollection.LANDSAT_ETM_L2
DataCollection.LANDSAT_OT_L1
DataCollection.LANDSAT_OT_L2
DataCollection.SENTINEL5P
DataCollection.SENTINEL3_OLCI
DataCollection.SENTINEL3_SLSTR
DataCollection.HARMONIZED_LANDSAT_SENTINEL


In [5]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

# Create a session
client = BackendApplicationClient(client_id=config.sh_client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                          client_secret=config.sh_client_secret, 
                          include_client_id=True)

# All requests using this session will have an access token automatically added
resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")
print(resp.content)

b'[]'


# Get collections

In [6]:
# Get collections
url = "https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections"
response = oauth.get(url)

In [7]:
if response.status_code == 200:
    catalog=response.json()

In [8]:
for collection in catalog['collections']:
    print(f"{collection['id']:<20}: {collection['description']}")

sentinel-2-l1c      : Sentinel 2 imagery processed to level 1C
sentinel-3-olci-l2  : Sentinel 3 data derived from imagery captured by OLCI sensor
sentinel-3-olci     : Sentinel 3 imagery captured by OLCI sensor
sentinel-3-slstr    : Sentinel 3 imagery captured by SLSTR sensor
sentinel-1-grd      : Sentinel 1 Ground Range Detected Imagery
sentinel-2-l2a      : Sentinel 2 imagery processed to level 2A
sentinel-5p-l2      : Sentinel 5 Precursor imagery captured by TROPOMI sensor


# Inspect a collection

In [9]:
collection='sentinel-2-l2a'
url = f"https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/{collection}/"
response = oauth.get(url)

In [10]:
if response.status_code == 200:
    catalog=response.json()

In [11]:
for band in catalog["summaries"]["eo:bands"]:
    print(band)

{'name': 'B01', 'common_name': 'coastal', 'center_wavelength': 0.4427, 'full_width_half_max': 0.021}
{'name': 'B02', 'common_name': 'blue', 'center_wavelength': 0.4924, 'full_width_half_max': 0.066}
{'name': 'B03', 'common_name': 'green', 'center_wavelength': 0.5598, 'full_width_half_max': 0.036}
{'name': 'B04', 'common_name': 'red', 'center_wavelength': 0.6646, 'full_width_half_max': 0.031}
{'name': 'B05', 'center_wavelength': 0.7041, 'full_width_half_max': 0.015}
{'name': 'B06', 'center_wavelength': 0.7405, 'full_width_half_max': 0.015}
{'name': 'B07', 'center_wavelength': 0.7828, 'full_width_half_max': 0.02}
{'name': 'B08', 'common_name': 'nir', 'center_wavelength': 0.8328, 'full_width_half_max': 0.106}
{'name': 'B8A', 'common_name': 'nir08', 'center_wavelength': 0.8647, 'full_width_half_max': 0.021}
{'name': 'B09', 'common_name': 'nir09', 'center_wavelength': 0.9451, 'full_width_half_max': 0.02}
{'name': 'B11', 'common_name': 'swir16', 'center_wavelength': 1.6137, 'full_width_half_

In [ ]:
print(json.dumps(catalog, indent=4))

# Search within a collection

In [13]:
bbox = BBox((3924000,3081000,3937000,3094000), crs=CRS(3035)).transform(CRS.WGS84)
bbox=[bbox.min_x,bbox.min_y,bbox.max_x,bbox.max_y]

date_start="2024-01-01"
date_end="2024-12-31"

collection='sentinel-2-l2a'

data = {
    "bbox": bbox,
    "datetime": f"{date_start}T00:00:00Z/{date_end}T23:59:59Z",
    "collections": [collection],
    "limit": 5,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/search"
response = oauth.post(url, json=data)

In [14]:
print(response.status_code)

200


In [ ]:
print(json.dumps(response.json(), indent=4))

# Search a BYOC

In [16]:
# "36caf873-32ca-44be-ba3e-e6c1b759211c" communiqué lors des présentations d'Eurostat
byoc_id="36caf873-32ca-44be-ba3e-e6c1b759211c"
date_start="2000-01-01"
date_end="2024-12-31"
data = {
    "bbox": bbox,
    "datetime": f"{date_start}T00:00:00Z/{date_end}T23:59:59Z",
    "collections": [f"byoc-{byoc_id}"],
    "limit": 5,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/search"
response = oauth.post(url, json=data)

In [17]:
if response.status_code == 200:
    catalog=response.json()
else:
    print(f'ERROR: {response.status_code}')

In [18]:
import json

print(json.dumps(catalog, indent=4))

{
    "type": "FeatureCollection",
    "features": [
        {
            "stac_version": "1.0.0",
            "stac_extensions": [
                "https://stac-extensions.github.io/projection/v1.0.0/schema.json"
            ],
            "id": "6a7f627b-90d8-47f9-8af4-a0263a6d1f2e",
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "crs": {
                    "type": "name",
                    "properties": {
                        "name": "urn:ogc:def:crs:OGC::CRS84"
                    }
                },
                "coordinates": [
                    [
                        [
                            -23.559654320106844,
                            24.793476852584043
                        ],
                        [
                            32.20174584528219,
                            28.624308253511483
                        ],
                        [
                            59.9057652170866